# TEST OF MODEL

In [ ]:
import tensorflow as tf
import numpy as np
import cv2

# ----------------------------
# Load model
# ----------------------------
model_path="D:\FINAL_YEAR_PROJECTS\Deep_face\88_ACCURACY\DeepFake-web\model_real\model.keras"
model = tf.keras.models.load_model(
    model_path
)

SEQ_LEN = 24
IMG_SIZE = 224

# ----------------------------
# Video reader (SAFE)
# ----------------------------
def read_video_for_test(video_path, seq_len=24):
    cap = cv2.VideoCapture(video_path)
    total = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    if total <= 0:
        cap.release()
        raise ValueError("❌ Unable to read video")

    frames = []

    idxs = np.linspace(0, total - 1, min(seq_len, total)).astype(int)

    for i in idxs:
        cap.set(cv2.CAP_PROP_POS_FRAMES, i)
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (IMG_SIZE, IMG_SIZE))
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frames.append(frame)

    cap.release()

    # 🔁 PAD if video is short
    while len(frames) < seq_len:
        frames.append(frames[-1])

    frames = np.array(frames, dtype=np.float32) / 255.0
    return frames


# ----------------------------
# Predict function
# ----------------------------
def predict_video(video_path):
    frames = read_video_for_test(video_path, SEQ_LEN)
    frames = np.expand_dims(frames, axis=0)  # (1, 24, 224, 224, 3)

    prediction = model.predict(frames, verbose=0)[0][0]

    print(f"\n🔍 Confidence Score: {prediction:.4f}")

    if prediction > 0.5:
        print("🔵 RESULT: REAL VIDEO")
    else:
        print("🔴 RESULT: FAKE VIDEO")


# ----------------------------
# Test
# ----------------------------
video_path = r"D:\FINAL_YEAR_PROJECTS\Deep_face\88_ACCURACY\DeepFake-web\id0_id1_0003.mp4"
predict_video(video_path)
